# Introduction

In the realm of genomics, understanding the distinct genetic variations among different species is crucial. One application of this understanding is the classification of DNA sequences, which can, for instance, differentiate between human and gorilla DNA. This analysis aims to investigate various machine learning approaches to classify given DNA sequences into one of these two categories, leveraging a dataset provided in the Kaggle competition, [ML Olympiad - Genome Sequences Classification](https://www.kaggle.com/competitions/ml-olympiad-gdscuiz-and-tfugagadir/data).

The dataset comprises sequences of DNA, each labeled as either 'human' or 'gorilla'. The primary goal is to build and compare models that can accurately predict these labels based on the DNA sequences. Five different approaches will be explored:
- Convolutional Neural Networks (CNN)
- Pre-trained BERT model specialized for genomic sequences
- Long Short-Term Memory networks (LSTM)
- Random Forest
- Multi Layer Perceptron (MLP)

Through this analysis, we aim to discern which model or combination of models yields the highest accuracy in classifying the DNA sequences. Additionally, we will look into potential improvements and optimizations to enhance the performance of the chosen models.

---

# Dataset
The dataset provided in the [ML Olympiad - Genome Sequences Classification competition](https://www.kaggle.com/competitions/ml-olympiad-gdscuiz-and-tfugagadir/data) consists of DNA sequences labeled as either 'human' or 'gorilla'. In this section, we'll explore the dataset to understand its structure, size, and the distribution of labels. We'll also conduct some preliminary analysis to identify any potential challenges or considerations for the subsequent modeling steps.

In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv('../input/ml-olympiad-gdscuiz-and-tfugagadir/train.csv')

# Display the first few rows of the dataset
data.head()

,Unnamed: 0,id,genome_sequence,species
0,0,11408003,ccacatcccctccagcacctgttgtttcctgactttttaatgattg...,Gorilla_gorilla
1,1,18639873,tgtttacttgccaatctttgtttagctgtcagagtggcttgctaaa...,Gorilla_gorilla
2,2,9869298,tctgtgaagaaagacattggtagcttgatggggatgacattgaatc...,Homo_sapiens
3,3,10762804,ttgtgagaattacgtgagatgatagatttagggactatagaatagt...,Gorilla_gorilla
4,4,13724428,gcaaaaaataagttgataagttgattgatatgttattagcttaatt...,Gorilla_gorilla


Let's lower the row count to speed up the analysis, as we won't use that much data anyway.

In [2]:
data = data.sample(n=1000000)

## Dataset Structure
Let's start by examining the structure of the dataset including the number of samples, features, and the distribution of labels.

In [7]:
# Getting the shape of the dataset
dataset_shape = data.shape
print(f'The dataset contains {dataset_shape[0]} samples and {dataset_shape[1]} columns.')

# Checking the distribution of labels
label_distribution = data['species'].value_counts(normalize=True)
label_distribution

The dataset contains 1000000 samples and 4 columns.


species
Homo_sapiens       0.500214
Gorilla_gorilla    0.499786
Name: proportion, dtype: float64

## Preliminary Analysis
We'll conduct some preliminary analysis to better understand the characteristics of the DNA sequences. This includes examining the length of the sequences, the distribution of nucleotide bases (A, C, G, T), and any missing or anomalous values.

In [9]:
# Checking for missing values
missing_values = data.isnull().sum()
missing_values

Unnamed: 0         0
id                 0
genome_sequence    0
species            0
dtype: int64

In [10]:
# Exploring the length of DNA sequences
sequence_lengths = data['genome_sequence'].apply(len)
sequence_lengths.describe()

count    1000000.000000
mean          79.999977
std            0.023000
min           57.000000
25%           80.000000
50%           80.000000
75%           80.000000
max           80.000000
Name: genome_sequence, dtype: float64

In [17]:
# Function to count the occurrences of each nucleotide in a sequence
# Concatenating all sequences into a single string
all_sequences = ''.join(data['genome_sequence'])

# Counting the occurrences of each nucleotide
base_counts = pd.Series({'a': all_sequences.count('a'), 
                         'c': all_sequences.count('c'), 
                         'g': all_sequences.count('g'), 
                         't': all_sequences.count('t')})

# Displaying the counts
base_counts

a    23912760
c    16068705
g    15967064
t    24051448
dtype: int64

The preliminary analysis provides insight into the basic characteristics of the DNA sequences in the dataset. The findings from this section will inform the choice and configuration of machine learning models in the subsequent steps of this analysis.

---

# Approaches
In this section, we explore various machine learning approaches to classify the DNA sequences as either human or gorilla. The models chosen for this analysis span a range of complexities and methodologies, from traditional machine learning to deep learning architectures. The objective is to compare the performance and insights gleaned from each model, thereby identifying the most effective strategy for this classification task. The approaches considered include Convolutional Neural Networks (CNN), a genomic sequences pre-trained BERT model, Long Short-Term Memory networks (LSTM), Random Forest, and Multi Layer Perceptron (MLP).

## Convolutional Neural Networks (CNN)
Convolutional Neural Networks (CNN) are particularly adept at identifying patterns in spatial or temporal data, making them a suitable choice for sequence data like DNA sequences. The convolution layers can detect motifs in the DNA sequences which can be crucial for accurate classification.

In [ ]:
# Placeholder for CNN model code

---

## Genomic Sequences Pre-trained BERT
The Bidirectional Encoder Representations from Transformers (BERT) model has shown promise in various NLP tasks. A version of BERT pre-trained on genomic sequences can potentially capture the contextual relationships between nucleotides in DNA sequences, thereby aiding in accurate classification.

In [ ]:
# Placeholder for genomic sequences pre-trained BERT model code

---

## Long Short-Term Memory Networks (LSTM)
Long Short-Term Memory networks (LSTM) are a type of recurrent neural network capable of learning long-term dependencies in data. Given the sequential nature of DNA, LSTMs can be employed to capture the inherent dependencies between nucleotides over varying sequence lengths.

In [12]:
# Placeholder for LSTM model code

---

## Random Forest
Random Forest is a versatile and robust machine learning algorithm capable of handling a mix of data types. By encoding the DNA sequences appropriately, Random Forest can be utilized to identify the distinguishing features between human and gorilla DNA.

In [3]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [23]:
import optuna
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import HashingVectorizer

In [35]:
data_1000 = data.sample(n=1000000//10)

In [36]:
# Getting the shape of the dataset
dataset_shape = data_1000.shape
print(f'The dataset contains {dataset_shape[0]} samples and {dataset_shape[1]} columns.')

# Checking the distribution of labels
label_distribution = data_1000['species'].value_counts(normalize=True)
label_distribution

The dataset contains 100000 samples and 4 columns.


species
Homo_sapiens       0.50028
Gorilla_gorilla    0.49972
Name: proportion, dtype: float64

In [37]:
# Define a function to convert a DNA sequence into overlapping k-mers
def kmerizer(seq, k=6):
    return [seq[i:i+k] for i in range(len(seq) - k + 1)]

# Test the function with a single DNA sequence
# print(get_kmers(data_1000['genome_sequence'][0]))

hash_vectorizer = HashingVectorizer(analyzer=kmerizer, n_features=2**20)  # Adjust n_features to manage memory usage

# Apply the vectorizer to the genome_sequence column of the DataFrame
X = hash_vectorizer.fit_transform(data_1000['genome_sequence'])

# Encode species labels
data_1000['species_encoded'] = data_1000['species'].map({'Homo_sapiens': 0, 'Gorilla_gorilla': 1})
y = data_1000['species_encoded']

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
def objective(trial):
    # Hyperparameters to be optimized
    n_estimators = trial.suggest_int('n_estimators', 10, 200)  # Expanded range
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss'])
    max_depth = trial.suggest_int('max_depth', 2, 32, log=True)
    min_samples_split = trial.suggest_float('min_samples_split', 0.1, 1)
    min_samples_leaf = trial.suggest_float('min_samples_leaf', 0.1, 0.5)
    min_weight_fraction_leaf = trial.suggest_float('min_weight_fraction_leaf', 0.0, 0.5)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 128, log=True)
    min_impurity_decrease = trial.suggest_float('min_impurity_decrease', 0.0, 1.0)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])

    classifier = RandomForestClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        min_weight_fraction_leaf=min_weight_fraction_leaf,
        max_features=max_features,
        max_leaf_nodes=max_leaf_nodes,
        min_impurity_decrease=min_impurity_decrease,
        bootstrap=bootstrap,
        random_state=50
    )

    classifier.fit(X_train, y_train)
    predictions = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    return accuracy

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Results of the optimization
print(f'Number of finished trials: {len(study.trials)}')
print(f'Best trial: {study.best_trial.params}')

[I 2023-10-14 16:32:36,549] A new study created in memory with name: no-name-79f3cd72-899e-40a2-b59b-bcd6eb355275
[I 2023-10-14 16:32:39,620] Trial 0 finished with value: 0.49965 and parameters: {'n_estimators': 158, 'criterion': 'log_loss', 'max_depth': 17, 'min_samples_split': 0.3880243171996576, 'min_samples_leaf': 0.4247838122008928, 'min_weight_fraction_leaf': 0.170287707952885, 'max_features': 'log2', 'max_leaf_nodes': 7, 'min_impurity_decrease': 0.6467542020296969, 'bootstrap': True}. Best is trial 0 with value: 0.49965.
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
[I 2023-10-14 16:32:40,715] Trial 1 finished with value: 0.49965 and parameters: {'n_estimators': 52, 'criterion': '

Number of finished trials: 100
Best trial: {'n_estimators': 158, 'criterion': 'log_loss', 'max_depth': 17, 'min_samples_split': 0.3880243171996576, 'min_samples_leaf': 0.4247838122008928, 'min_weight_fraction_leaf': 0.170287707952885, 'max_features': 'log2', 'max_leaf_nodes': 7, 'min_impurity_decrease': 0.6467542020296969, 'bootstrap': True}


In [44]:
# Check the dimensions and the type of the feature matrix
print(X.shape, type(X))

(100000, 1048576) <class 'scipy.sparse.csr.csr_matrix'>


In [45]:
# Assumez que best_params est le dictionnaire des meilleurs paramètres retournés par Optuna
best_params = study.best_trial.params

# Créez un nouveau classificateur Random Forest avec ces paramètres
final_classifier = RandomForestClassifier(**best_params, random_state=50, verbose=1)

# Entraînez le classificateur sur l'ensemble d'entraînement
final_classifier.fit(X_train, y_train)

# Faites des prédictions sur l'ensemble de test
final_predictions = final_classifier.predict(X_test)

# Évaluez la performance du classificateur
final_accuracy = accuracy_score(y_test, final_predictions)
conf_matrix = confusion_matrix(y_test, final_predictions)
class_report = classification_report(y_test, final_predictions)

# Print les métriques d'évaluation
print(f'Final Accuracy: {final_accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.5s


Final Accuracy: 0.49965
Confusion Matrix:
[[ 9993     0]
 [10007     0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.50      1.00      0.67      9993
           1       0.00      0.00      0.00     10007

    accuracy                           0.50     20000
   macro avg       0.25      0.50      0.33     20000
weighted avg       0.25      0.50      0.33     20000



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


---

## Multi Layer Perceptron (MLP)
Multi Layer Perceptron (MLP) is a class of feedforward artificial neural network. By flattening the DNA sequences and employing a suitable encoding scheme, MLPs can be trained to differentiate between human and gorilla DNA based on the input features.

In [ ]:
# Placeholder for MLP model code

---
---

In [4]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import numpy as np

from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import TrainingArguments, Trainer
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load dataset
data = pd.read_csv('../input/ml-olympiad-gdscuiz-and-tfugagadir/train.csv',usecols=['genome_sequence','species'])
data = data.sample(n=10000)
sequences = data['genome_sequence'].tolist()
labels = data['species'].tolist()

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
inputs = tokenizer(sequences, truncation=True, padding=True, return_tensors='tf')

In [24]:
# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Convert input_ids to numpy array before splitting
input_ids_np = inputs['input_ids'].numpy()

# Split data
train_input_ids, val_input_ids, train_labels, val_labels = train_test_split(input_ids_np, encoded_labels, test_size=0.2)

# If you need to convert them back to tensors for training:
train_input_ids = tf.convert_to_tensor(train_input_ids)
val_input_ids = tf.convert_to_tensor(val_input_ids)

In [29]:
# Load pre-trained model and modify
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Training arguments and training
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=50,
    learning_rate=2e-5,
)

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Define loss and optimizer
loss = SparseCategoricalCrossentropy(from_logits=True)
optimizer = Adam()

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Convert labels to tensors
train_labels_tensor = tf.convert_to_tensor(train_labels)
val_labels_tensor = tf.convert_to_tensor(val_labels)

# Train the model
model.fit(train_input_ids, train_labels_tensor, validation_data=(val_input_ids, val_labels_tensor), epochs=3, batch_size=8)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
1000/1000 [==============================] - 117s 72ms/step - loss: 0.7013 - accuracy: 0.4992 - val_loss: 0.6950 - val_accuracy: 0.4965
Epoch 2/3
1000/1000 [==============================] - 68s 68ms/step - loss: 0.7008 - accuracy: 0.4976 - val_loss: 0.6977 - val_accuracy: 0.4965
Epoch 3/3
1000/1000 [==============================] - 69s 69ms/step - loss: 0.6992 - accuracy: 0.4976 - val_loss: 0.6941 - val_accuracy: 0.4965


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score

# Load the test dataset
test_data = pd.read_csv('test.csv')
test_sequences = test_data['genome_sequence'].tolist()

# Tokenize the sequences
test_inputs = tokenizer(test_sequences, truncation=True, padding=True, return_tensors='tf')

# Predict using the fine-tuned model
predictions = model(test_inputs).logits
predicted_labels = predictions.argmax(axis=1)

# Convert species names in the test set to numerical labels
true_labels = label_encoder.transform(test_data['species'].tolist())

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Load the data
train_data = pd.read_csv('../input/ml-olympiad-gdscuiz-and-tfugagadir/train.csv',usecols=['genome_sequence','species'])
train_data = train_data.sample(n=500000)
species_counts = train_data['species'].value_counts(normalize=True)
print(species_counts)

In [ ]:
# Build the model
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu',
                           input_shape=(9, 9, 4),
                           padding='same',
                           kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', 
                           padding='same',
                           kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', 
                           padding='same',
                           kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation='softmax')  # Assuming 2 classes, adjust if necessary
])

model.summary()

In [ ]:
def preprocess_dataset(data, test=False):
    def one_hot_encode(sequence):
        mapping = {'a': [1, 0, 0, 0], 'c': [0, 1, 0, 0], 'g': [0, 0, 1, 0], 't': [0, 0, 0, 1]}
        return [mapping[char] for char in sequence]

    encoded_seqs = data['genome_sequence'].apply(one_hot_encode)
    padded_seqs = tf.keras.preprocessing.sequence.pad_sequences(encoded_seqs, padding='post')
    
    if not test:
        # Encode the species labels into numerical values
        label_encoder = LabelEncoder()
        encoded_labels = label_encoder.fit_transform(data['species'])
        encoded_labels = tf.keras.utils.to_categorical(encoded_labels, num_classes=2)
    
    num_samples = len(padded_seqs)
    reshaped_data = np.zeros((num_samples, 9, 9, 4))  # Initialize an empty array of the desired shape

    for i, sequence in enumerate(padded_seqs):
        # Reshape the sequence to a 2D 80x4 matrix
        sequence_2d = sequence.reshape(-1, 4)

        # Pad this matrix with a row of zeros to get a 81x4 matrix
        padded_sequence = np.vstack((sequence_2d, np.zeros((1, 4))))

        # Reshape this matrix to a 9x9x4 tensor
        reshaped_sequence = padded_sequence.reshape(9, 9, 4)

        # Store the reshaped sequence in the reshaped_data array
        reshaped_data[i] = reshaped_sequence
    if not test:
        return reshaped_data, encoded_labels
    else:
        return reshaped_data

In [ ]:
x_train_data, y_train_data = preprocess_dataset(train_data)

In [ ]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_train_data, y_train_data, test_size=0.2, random_state=42)

In [ ]:
def plot_images(data_piece):
    fig, axs = plt.subplots(1, 4, figsize=(20, 5))
    
    # Assuming data_piece has shape (9, 9, 4)
    for i in range(4):
        img = data_piece[:, :, i]
        axs[i].imshow(img, cmap='gray')  # or choose a different colormap if preferred
        axs[i].axis('off')  # to remove the axes for clarity
        axs[i].set_title(f'Channel {i+1}')
    
    plt.show()
plot_images(x_train[0])

In [ ]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define the Early Stopping and Reduce LR On Plateau callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001, verbose=1)

In [ ]:
# Train the model with the callbacks
history = model.fit(x_train, y_train,
                    validation_data=(x_test, y_test),
                    epochs=25,
                    batch_size=128,
                    callbacks=[early_stopping, reduce_lr])

In [ ]:
# Plotting the training and validation accuracy
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

# Plotting the training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
test_data = pd.read_csv('../input/ml-olympiad-gdscuiz-and-tfugagadir/test.csv', usecols=['genome_sequence'])

x_test_data = preprocess_dataset(test_data, True)

# Make predictions
predictions = model.predict(x_test_data)
predicted_labels = np.argmax(predictions, axis=1)  # assuming a multi-class classification problem
di = {
    0: "Homo_sapiens",
    1: "Gorilla_gorilla"
}
predicted_labels = [v for v in predicted_labels]

In [ ]:
np.unique(predicted_labels, return_counts=True)

In [ ]:
# Create submission DataFrame
submission_df = pd.DataFrame({'id': test_data.index, 'species': predicted_labels})

# Save to CSV
submission_df.to_csv('submission.csv', index=False)